In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

print("Bibliotecas importadas")

In [ ]:
target_column = None
for col in ['status', 'labels']:
    if col in train_data.columns:
        target_column = col
        break

print(f"Target: '{target_column}'")

print(f"\nResumo: {len(train_data)} startups, taxa sucesso: {train_data[target_column].mean():.2%}")

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler

numeric_cols = X.select_dtypes(include=[np.number]).columns
correlations = {}
for col in numeric_cols:
    if X[col].std() > 0:  # Evitar divisão por zero
        corr = np.corrcoef(X[col], y)[0, 1]
        correlations[col] = abs(corr) if not np.isnan(corr) else 0

print("Top 10 features por correlação com target:")
sorted_corr = sorted(correlations.items(), key=lambda x: x[1], reverse=True)
for i, (feature, corr) in enumerate(sorted_corr[:10]):
    print(f"{i+1:2d}. {feature}: {corr:.3f}")

k_best = SelectKBest(score_func=f_classif, k=min(20, X.shape[1]))
X_selected = k_best.fit_transform(X, y)
X_test_selected = k_best.transform(X_test)

selected_features = X.columns[k_best.get_support()].tolist()
print(f"\nFeatures selecionadas ({len(selected_features)}):")
for i, feature in enumerate(selected_features):
    print(f"{i+1:2d}. {feature}")

X = pd.DataFrame(X_selected, columns=selected_features)
X_test = pd.DataFrame(X_test_selected, columns=selected_features)

print(f"\nDados finais - Treino: {X.shape}, Teste: {X_test.shape}")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

modelo = GradientBoostingClassifier(random_state=42, n_estimators=100, learning_rate=0.1, max_depth=3)
modelo.fit(X_train, y_train)

pred_val = modelo.predict(X_val)
precisao = accuracy_score(y_val, pred_val)
precision = precision_score(y_val, pred_val)
recall = recall_score(y_val, pred_val)
f1 = f1_score(y_val, pred_val)

print(" AVALIAÇÃO DO MODELO ")
print(f"Acurácia:  {precisao:.3f} ({precisao:.1%})")
print(f"Precisão:  {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"F1-Score:  {f1:.3f}")

print(f"\nRelatório de Classificação:")
print(classification_report(y_val, predicoes_val, target_names=['Falha', 'Sucesso']))

feature_importance = pd.DataFrame({
    'feature': selected_features,
    'importance': modelo.feature_importances_
}).sort_values('importance', ascending=False)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 150, 200],
    'learning_rate': [0.05, 0.1, 0.15],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10]
}

print("Iniciando busca de hiperparâmetros...")
print(f"Testando {len(param_grid['n_estimators']) * len(param_grid['learning_rate']) * len(param_grid['max_depth']) * len(param_grid['min_samples_split'])} combinações")

grid_search = GridSearchCV(
    estimator=GradientBoostingClassifier(random_state=42),
    param_grid=param_grid,
    cv=3,  # 3-fold cross validation
    scoring='accuracy',
    n_jobs=-1,  # Usar todos os cores disponíveis
    verbose=1
)

grid_search.fit(X_train, y_train)

print(f"\nMelhores parâmetros encontrados:")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")

print(f"\nMelhor score CV: {grid_search.best_score_:.4f}")

modelo_otimizado = grid_search.best_estimator_
modelo_otimizado.fit(X_train, y_train)

pred_otimizado = modelo_otimizado.predict(X_val)
acc_otimizado = accuracy_score(y_val, pred_otimizado)
precision_otimizado = precision_score(y_val, pred_otimizado)
recall_otimizado = recall_score(y_val, pred_otimizado)
f1_otimizado = f1_score(y_val, pred_otimizado)

print(f"\n COMPARAÇÃO DE MODELOS ")
print(f"Modelo Inicial  - Acurácia: {precisao:.3f}, F1: {f1:.3f}")
print(f"Modelo Otimizado - Acurácia: {acc_otimizado:.3f}, F1: {f1_otimizado:.3f}")

if acc_otimizado > precisao:
    print(f"Melhoria de {(acc_otimizado - precisao):.3f} pontos na acurácia!")
    modelo_final = modelo_otimizado
    precisao_final = acc_otimizado
else:
    print("Modelo inicial teve melhor performance")
    modelo_final = modelo
    precisao_final = precisao

print(f"\nModelo final - Acurácia: {precisao_final:.1%}")

In [ ]:
print("Retreinando modelo final com todos os dados...")

if 'modelo_final' in locals():
    modelo_submissao = modelo_final.__class__(**modelo_final.get_params())
else:
    modelo_submissao = GradientBoostingClassifier(random_state=42, n_estimators=150, learning_rate=0.1, max_depth=4)

modelo_submissao.fit(X, y)

predicoes_teste = modelo_submissao.predict(X_test)
probabilidades_teste = modelo_submissao.predict_proba(X_test)[:, 1]

unique, counts = np.unique(predicoes_teste, return_counts=True)
print(f"\nDistribuição das previsões:")
for val, count in zip(unique, counts):
    resultado = "Sucesso" if val == 1 else "Falha"
    porcentagem = (count / len(predicoes_teste)) * 100
    print(f"   {resultado}: {count:3d} startups ({porcentagem:5.1f}%)")

print(f"\nAnálise de confiança:")
print(f"Probabilidade média para classe 1: {probabilidades_teste.mean():.3f}")
print(f"Previsões com alta confiança (>0.8): {sum(probabilidades_teste > 0.8):3d}")
print(f"Previsões com baixa confiança (<0.2): {sum(probabilidades_teste < 0.2):3d}")

submissao = sample_submission.copy()
submissao[target_column] = predicoes_teste
submissao.to_csv('minha_submissao.csv', index=False)

print(f"\nArquivo 'minha_submissao.csv' criado com {len(predicoes_teste)} previsões")

In [ ]:
print("=" * 50)
print("RESUMO FINAL DO PROJETO")
print("=" * 50)

print(f"\nDADOS:")
print(f"   Startups de treino: {len(train_data):,}")
print(f"   Startups de teste: {len(test_data):,}")
print(f"   Features selecionadas: {X.shape[1]}")
print(f"   Taxa de sucesso no treino: {y.mean():.1%}")

if 'precisao_final' in locals():
    accuracy_final = precisao_final
else:
    accuracy_final = precisao

print(f"\nMODELO:")
print(f"   Algoritmo: Gradient Boosting Classifier")
print(f"   Acurácia final: {accuracy_final:.1%}")
if 'precision_otimizado' in locals():
    print(f"   Precisão: {precision_otimizado:.3f}")
    print(f"   Recall: {recall_otimizado:.3f}")
    print(f"   F1-Score: {f1_otimizado:.3f}")


if accuracy_final >= 0.8:
    print(f"   [OK] Acurácia >= 80%")
    status_final = "APROVADO"

print(f"\nTOP 5 FEATURES MAIS IMPORTANTES:")
if 'feature_importance' in locals():
    for i, row in feature_importance.head(5).iterrows():
        print(f"   {i+1}. {row['feature']}: {row['importance']:.4f}")

success_rate = (predicoes_teste == 1).sum() / len(predicoes_teste)
print(f"\nPREVISÕES:")
print(f"   Taxa de sucesso prevista: {success_rate:.1%}")

